In [0]:
%matplotlib inline
import math
import numpy as np
import os
import random
import tensorflow as tf
from matplotlib import pylab
from collections import Counter
import csv

# Seq2Seq Items
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.python.layers.core import Dense
import nltk

In [0]:
import csv
import nltk as nlp
from nltk import word_tokenize
import string
import nltk
import re

summaries = []
texts = []

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    return filter(lambda x: x in printable, text) #filter funny characters, if any. 
  
def remove_html(text):
  tag_re = re.compile(r'(<!--.*?-->|<[^>]*>)')
  no_tags = tag_re.sub('', text)
  return no_tags

In [23]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


gpg: keybox '/tmp/tmp7qc5fymv/pubring.gpg' created
gpg: /tmp/tmp7qc5fymv/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


KeyboardInterrupt: ignored

In [24]:
!mkdir -p drive
!google-drive-ocamlfuse drive


fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [4]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:

n_summaries =[]
n_texts = []
with open('drive/Colab Notebooks/Tutorial/Data/Reviews.csv', 'rb') as csvfile: #Data from https://www.kaggle.com/snap/amazon-fine-food-reviews
    Reviews = csv.DictReader(csvfile)
    for row in Reviews:
        clean_text = clean(row['Text'])
        #new_text = remove_html(clean_text)
        clean_summary = clean(row['Summary'])
        #new_summary = remove_html(clean_summary)
        n_summaries.append(word_tokenize(clean_summary))
        n_texts.append(word_tokenize(clean_text))

FileNotFoundError: ignored

In [0]:
final_texts = []
for i in range(len(texts)):
  if i < 100:
    final_texts.append(texts[i])
  

In [0]:
final_summaries = []

for i in range(len(summaries)):
  if i < 100:
    final_summaries.append(summaries[i])
    
    

In [18]:
len(final_summaries)

0

In [0]:
!pip install simplejson

^C
Traceback (most recent call last):
  File "/usr/bin/pip2", line 9, in <module>
    from pip import main
  File "/usr/lib/python2.7/dist-packages/pip/__init__.py", line 21, in <module>
    from pip._vendor.requests.packages.urllib3.exceptions import DependencyWarning
  File "/usr/lib/python2.7/dist-packages/pip/_vendor/__init__.py", line 76, in <module>
    vendored("pkg_resources")
  File "/usr/lib/python2.7/dist-packages/pip/_vendor/__init__.py", line 36, in vendored
    __import__(modulename, globals(), locals(), level=0)
  File "/usr/share/python-wheels/pkg_resources-0.0.0-py2.py3-none-any.whl/pkg_resources/__init__.py", line 16, in <module>
KeyboardInterrupt


In [0]:
import simplejson


In [0]:
vocab = set()
for line in final_texts:
  for token in line:
    vocab.add(token)

In [0]:
count_dictionary = {} 

for line in final_texts:
  for token in line:
    if token not in count_dictionary:
      count_dictionary[token] = 1
    else:
      count_dictionary[token] += 1

In [0]:
target_count_dictionary = {}

for line in final_summaries:
  for token in line:
    if token not in target_count_dictionary:
      target_count_dictionary[token] = 1
    else:
      target_count_dictionary[token] += 1
      

In [3]:
source_sents = []
target_sents = []

for line in final_texts:
  new_line = []
  new_line.append("<s>")
  for token in line:
    if count_dictionary[token] < 2:
      word = "<UNK>"
    else:
      word = token
    new_line.append(word)
  new_line.append("</s>")
  source_sents.append(new_line)
  
for line in final_summaries:
  new_line = []
  new_line.append("<s>")
  for token in line:
    if target_count_dictionary[token] < 2:
      word = "<UNK>"
    else:
      word = token
    new_line.append(word)
  new_line.append("</s>")
  target_sents.append(new_line)

NameError: ignored

In [0]:
# with open("drive/Colab Notebooks/Summarization/source.txt", "w") as fp:
#   simplejson.dump(source_sents, fp)
  

KeyboardInterrupt: ignored

In [0]:
# with open("drive/Colab Notebooks/Summarization/target.txt", "w") as fp:
#   simplejson.dump(target_sents, fp)
  

In [2]:
src_dictionary = {}

index = 0

for line in source_sents:
  for token in line:
    if token not in src_dictionary:
      src_dictionary[token] = index
      index += 1
    else:
      pass


NameError: ignored

In [0]:
tgt_dictionary = {}

index = 0
for line in target_sents:
  for token in line:
    if token not in tgt_dictionary:
      tgt_dictionary[token] = index
      index += 1
    else:
      pass
    

In [0]:
src_reverse_dictionary = dict(zip(src_dictionary.values(),src_dictionary.keys()))


In [0]:
tgt_reverse_dictionary = dict(zip(tgt_dictionary.values(),src_dictionary.keys()))


In [0]:
len(src_dictionary)

668

In [0]:
# np.savetxt('drive/Colab Notebooks/Summarization/source.txt', source_sents, delimiter='\n', fmt="%s")

In [0]:
# np.savetxt('drive/Colab Notebooks/Summarization/target.txt', target_sents, delimiter='\n', fmt="%s")

In [0]:
# np.save("drive/Colab Notebooks/Summarization/source_dict.npy", src_dictionary)
# np.save("drive/Colab Notebooks/Summarization/source_reverse_dict.npy", src_reverse_dictionary)
# np.save("drive/Colab Notebooks/Summarization/target_dict.npy", tgt_dictionary)
# np.save("drive/Colab Notebooks/Summarization/target_reverse_dict.npy", tgt_reverse_dictionary)


In [0]:
!pip2 install gensim

  Using cached gensim-3.4.0-cp27-cp27mu-manylinux1_x86_64.whl


In [0]:
import gensim
from gensim.models import word2vec

In [0]:
all_sentences = []

for line in source_sents:
  all_sentences.append(line)

for line in target_sents:
  all_sentences.append(line)

In [0]:
len(all_sentences)

200

In [0]:
model = word2vec.Word2Vec(all_sentences, iter=20, min_count=2, size=128, workers=4)


In [0]:
def create_embedding_matrix(model):
   # convert the wv word vectors into a numpy matrix that is suitable for insertion
   # into our TensorFlow and Keras models
   embedding_matrix = np.zeros((len(model.wv.vocab), 128), dtype=np.float32)
   for i in range(len(model.wv.vocab)):
       embedding_vector = model.wv[model.wv.index2word[i]]
       if embedding_vector is not None:
           embedding_matrix[i] = embedding_vector
   return embedding_matrix

In [0]:
embedding_matrix = create_embedding_matrix(model)
#embedding_matrix = np.load("drive/Colab Notebooks/Summarization/embedding_matrix.npy")

In [0]:
embedding_matrix.shape

(673, 128)

In [0]:
# np.save("drive/Colab Notebooks/Summarization/embedding_matrix.npy", embedding_matrix)

In [0]:
NUM_EPOCS = 2
RNN_STATE_DIM = 512
RNN_NUM_LAYERS = 2
ENCODER_EMBEDDING_DIM = DECODER_EMBEDDING_DIM = 128
 
BATCH_SIZE = int(32)
LEARNING_RATE = 0.0003
 
INPUT_NUM_VOCAB = len(src_dictionary)
OUTPUT_NUM_VOCAB = len(tgt_dictionary)

In [0]:
source_sents.pop(539)
target_sents.pop(539)

['<s>',
 '<UNK>',
 '<UNK>',
 'i',
 '<UNK>',
 '...',
 '<UNK>',
 '<UNK>',
 'i',
 '<UNK>',
 '...',
 '<UNK>',
 'o',
 'o',
 'r',
 'a',
 '<UNK>',
 '...',
 '.',
 '!',
 '!',
 '!',
 '!',
 '!',
 ':',
 ')',
 '(',
 'b',
 'a',
 '<UNK>',
 'k',
 '<UNK>',
 'a',
 'r',
 '<UNK>',
 '...',
 '<UNK>',
 'a',
 'r',
 'b',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 ')',
 '</s>']

In [0]:
length = []
for line in source_sents:
  length.append(len(line))
  
max_length_source = max(length)

In [0]:
max_length_source

472

In [0]:
length = []
for line in target_sents:
  length.append(len(line))
  
max_length_target = max(length)


print(max_length_target)

target_sents

15


[['<s>', 'good', '<UNK>', 'dog', 'food', '</s>'],
 ['<s>', 'not', 'as', '<UNK>', '</s>'],
 ['<s>', '``', '<UNK>', "''", '<UNK>', 'it', '<UNK>', '</s>'],
 ['<s>', '<UNK>', '<UNK>', '</s>'],
 ['<s>', 'great', 'taffy', '</s>'],
 ['<s>', '<UNK>', 'taffy', '</s>'],
 ['<s>',
  'great',
  '!',
  'just',
  'as',
  'good',
  'as',
  'the',
  'expensive',
  'brands',
  '!',
  '</s>'],
 ['<s>', '<UNK>', ',', '<UNK>', 'taffy', '</s>'],
 ['<s>', '<UNK>', '<UNK>', '</s>'],
 ['<s>', 'healthy', 'dog', 'food', '</s>'],
 ['<s>', 'the', 'best', 'hot', 'sauce', 'in', 'the', 'world', '</s>'],
 ['<s>',
  'my',
  'cats',
  'love',
  'this',
  '``',
  '<UNK>',
  "''",
  'food',
  'better',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  'food',
  '</s>'],
 ['<s>',
  'my',
  'cats',
  '<UNK>',
  'not',
  '<UNK>',
  'of',
  'the',
  '<UNK>',
  'food',
  '</s>'],
 ['<s>', 'fresh', 'and', '<UNK>', '!', '</s>'],
 ['<s>', 'strawberry', 'twizzlers', '-', 'yummy', '</s>'],
 ['<s>',
  '<UNK>',
  'of',
  'twizzlers',
  ',',
  'just

In [0]:
for i,j in enumerate(length):
  if j == 24:
    print(i)
  

477


In [0]:
target_sents[477]

['<s>',
 'fantastic',
 '!',
 'you',
 'can',
 'not',
 '<UNK>',
 'this',
 'taste',
 'and',
 'you',
 'can',
 'not',
 '<UNK>',
 'to',
 'it',
 'only',
 'one',
 '<UNK>',
 'you',
 'are',
 '<UNK>',
 '<UNK>',
 '</s>']

In [0]:
vocab_size = 14971
num_units = 128
input_size = 128
batch_size = 17

In [0]:
decoder_type = 'basic' # could be basic or attention


In [0]:
input_size = 128
encoder_emb_layer = embedding_matrix
decoder_emb_layer = embedding_matrix

In [0]:
# enc_train_inputs = []
# dec_train_inputs = []


# tf.reset_default_graph()


# enc_train_inputs = tf.placeholder(tf.int32, shape=[batch_size],name='enc_train_inputs')

# dec_train_inputs = tf.placeholder(tf.int32, shape=[batch_size],name='dec_train_inputs')
    
# encoder_emb_inp = tf.nn.embedding_lookup(encoder_emb_layer, enc_train_inputs)

# decoder_emb_inp = tf.nn.embedding_lookup(decoder_emb_layer, dec_train_inputs)

# enc_train_inp_lengths = tf.placeholder(tf.int32, shape=[batch_size],name='train_input_lengths')
# dec_train_inp_lengths = tf.placeholder(tf.int32, shape=[batch_size],name='train_output_lengths')


In [0]:
#   #############################################
#   ## Encoder
#   encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

#   initial_state = encoder_cell.zero_state(batch_size, dtype=tf.float32)

#   encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
#       encoder_cell, encoder_emb_inp, initial_state=initial_state,
#       sequence_length=enc_train_inp_lengths,
#       time_major=True, swap_memory=True)
  
#   # Build RNN cell
  

  
#   ###############################################
 
  
#   ############################################################

 


In [1]:
NUM_EPOCS = 2
RNN_STATE_DIM = 128
RNN_NUM_LAYERS = 1
ENCODER_EMBEDDING_DIM = DECODER_EMBEDDING_DIM = 128
 
BATCH_SIZE = int(32)
LEARNING_RATE = 0.0003
 
INPUT_NUM_VOCAB = len(src_dictionary)
OUTPUT_NUM_VOCAB = len(tgt_dictionary)

NameError: ignored

In [0]:
tf.reset_default_graph()

encoder_input_seq = tf.placeholder(
    tf.int32, 
    [None, None],  
    name='encoder_input_seq'
)

encoder_seq_len = tf.placeholder(
    tf.int32, 
    (None,), 
    name='encoder_seq_len'
)
 
# Decoder placeholders
decoder_output_seq = tf.placeholder( 
    tf.int32, 
    [None, None],
    name='decoder_output_seq'
)

decoder_seq_len = tf.placeholder(
    tf.int32,
    (None,), 
    name='decoder_seq_len'
)

max_decoder_seq_len = tf.reduce_max( 
    decoder_seq_len, 
    name='max_decoder_seq_len'
)

In [0]:
def make_cell(state_dim):
    lstm_initializer = tf.random_uniform_initializer(-0.1, 0.1)
    return tf.contrib.rnn.LSTMCell(state_dim, initializer=lstm_initializer)
 
def make_multi_cell(state_dim, num_layers):
    cells = [make_cell(state_dim) for _ in range(num_layers)]
    return tf.contrib.rnn.MultiRNNCell(cells)

In [0]:
# Encoder embedding
 
# encoder_input_embedded = tf.contrib.layers.embed_sequence(
#     encoder_input_seq,     
#     INPUT_NUM_VOCAB,        
#     ENCODER_EMBEDDING_DIM  
# )

encoder_emb_layer = embedding_matrix

encoder_input_embedded = tf.nn.embedding_lookup(encoder_emb_layer, encoder_input_seq)

 
## Encodering output
 
encoder_multi_cell = make_multi_cell(RNN_STATE_DIM, RNN_NUM_LAYERS)
 
encoder_output, encoder_state = tf.nn.dynamic_rnn(
    encoder_multi_cell, 
    encoder_input_embedded, 
    sequence_length=encoder_seq_len, 
    dtype=tf.float32
)
 
del(encoder_output)

In [0]:
decoder_raw_seq = decoder_output_seq[:, :-1]  

go_prefixes = tf.fill([BATCH_SIZE, 1], tgt_dictionary['<s>'])  

decoder_input_seq = tf.concat([go_prefixes, decoder_raw_seq], 1)

In [0]:
#decoder_embedding = tf.Variable(tf.random_uniform([OUTPUT_NUM_VOCAB, 
#                                                   DECODER_EMBEDDING_DIM]))

decoder_embedding = embedding_matrix  
decoder_input_embedded = tf.nn.embedding_lookup(decoder_embedding, 
                                                decoder_input_seq)

decoder_multi_cell = make_multi_cell(RNN_STATE_DIM, RNN_NUM_LAYERS)
 
output_layer_kernel_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
output_layer = Dense(
    OUTPUT_NUM_VOCAB,
    kernel_initializer = output_layer_kernel_initializer
)

In [0]:
with tf.variable_scope("decode"):
 
    training_helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=decoder_input_embedded,
        sequence_length=decoder_seq_len,
        time_major=False
    )
 
    training_decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_multi_cell,
        training_helper,
        encoder_state,
        output_layer
    ) 
 
    training_decoder_output_seq, _, _ = tf.contrib.seq2seq.dynamic_decode(
        training_decoder, 
        impute_finished=True, 
        maximum_iterations=max_decoder_seq_len
    )

In [0]:
with tf.variable_scope("decode", reuse=True):
    start_tokens = tf.tile(
        tf.constant([tgt_dictionary['<s>']], 
                    dtype=tf.int32), 
        [BATCH_SIZE], 
        name='start_tokens')
 
    # Helper for the inference process.
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embedding=decoder_embedding,
        start_tokens=start_tokens,
        end_token=tgt_dictionary['</s>']
    )
 
    # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_multi_cell,
        inference_helper,
        encoder_state,
        output_layer
    )
 
    # Perform dynamic decoding using the decoder
    inference_decoder_output_seq, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inference_decoder,
        impute_finished=True,
        maximum_iterations=max_decoder_seq_len
    )

In [0]:
training_logits = tf.identity(training_decoder_output_seq.rnn_output, name='logits')
inference_logits = tf.identity(inference_decoder_output_seq.sample_id, name='predictions')
 
# Create the weights for sequence_loss
masks = tf.sequence_mask(
    decoder_seq_len, 
    max_decoder_seq_len, 
    dtype=tf.float32, 
    name='masks'
)
 
cost = tf.contrib.seq2seq.sequence_loss(
    training_logits,
    decoder_output_seq,
    masks
)


In [0]:
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
train_pred = training_decoder_output_seq.sample_id

gradients = optimizer.compute_gradients(cost)
capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var)
                        for grad, var in gradients if grad is not None]
train_op = optimizer.apply_gradients(capped_gradients)

In [0]:
def pad(xs, size, pad):
    return xs + [pad] * (size - len(xs))

In [0]:
input_seq = [
    [src_dictionary.get(symbol, src_dictionary['<UNK>']) 
        for symbol in line]  
    for line in source_sents
]


In [0]:
output_seq = [
    [tgt_dictionary.get(symbol, tgt_dictionary['<UNK>']) 
        for symbol in line] + [tgt_dictionary['</s>']]  
    for line in target_sents  
]

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()  
 
for epoch in range(NUM_EPOCS + 1):  
    for batch_idx in range(len(source_sents) // BATCH_SIZE):         
        input_batch, input_lengths, output_batch, output_lengths = [], [], [], []
        for sentence in source_sents[batch_idx:batch_idx + BATCH_SIZE]:
            symbol_sent = [src_dictionary[symbol] for symbol in sentence]
            padded_symbol_sent = pad(symbol_sent, max_length_source, src_dictionary['</s>'])
            input_batch.append(padded_symbol_sent)
            input_lengths.append(len(sentence))
        for sentence in target_sents[batch_idx:batch_idx + BATCH_SIZE]:
            symbol_sent = [tgt_dictionary[symbol] for symbol in sentence]
            padded_symbol_sent = pad(symbol_sent, max_length_target, tgt_dictionary['</s>'])
            output_batch.append(padded_symbol_sent)
            output_lengths.append(len(sentence))

            
        _, cost_val, pred = sess.run( 
            [train_op, cost, train_pred] 
            feed_dict={
                encoder_input_seq: input_batch,
                encoder_seq_len: input_lengths,
                decoder_output_seq: output_batch,
                decoder_seq_len: output_lengths
            }
        )
        
        if batch_idx % 629 == 0:
            print('Epcoh {}. Batch {}/{}. Cost {}'.format(epoch, batch_idx, len(target_sents) // BATCH_SIZE, cost_val))

        saver.save(sess, 'drive/Colab Notebooks/Summarization/model.ckpt')   
sess.close()

Epcoh 0. Batch 0/3. Cost 4.26188230515


KeyboardInterrupt: ignored

In [0]:
sess = tf.InteractiveSession()    
saver.restore(sess, 'drive/Colab Notebooks/Summarization/model.ckpt')

example_input_sent = "this not a good dog food"
example_input_symb = [tgt_dictionary[symbol] for symbol in example_input_sent.split()]
example_input_batch = [pad(example_input_symb, max_length_source, tgt_dictionary['<s>'])] * BATCH_SIZE
example_input_lengths = [len(example_input_sent)] * BATCH_SIZE



output_ints = sess.run(inference_logits, feed_dict={
    encoder_input_seq: example_input_batch,
    encoder_seq_len: example_input_lengths,
    decoder_seq_len: example_input_lengths
})[0]

output_str = ' '.join([tgt_reverse_dictionary[i] for i in output_ints])
print(output_str)

INFO:tensorflow:Restoring parameters from drive/Colab Notebooks/Summarization/model.ckpt
non-instant non-instant worth
